> 2022/01

# Groupby : aplit-apply-combine

- https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html

"group by"는 아래 단계를 하나 혹은 그 이상 적용하는 것이다.

 - splitting: 어떤 분류를 기반으로 데이터를 그룹로 작성
 - applying: 각 개별 그룹에 적용하는 함수
 - combining: 결과를 구조화 한다.

pandas 객체는 모든 축(axis)에서 분할 할 수 있습니다. 그룹화의 추상적 인 정의는 레이블을 그룹 이름에 매핑하는 것입니다.

여기서는 **2020 시가총액 데이터** 을 사용하고 있다.

- Date : 날짜 (DatetimeIndex)
- Rank: 시가총액 순위 (당일)
- Code : 종목코드
- Name : 종명이름
- Open : 시가
- High : 고가
- Low : 저가
- Close : 종가
- Volume : 거래량
- Amount : 거래대금
- Changes : 전일대비
- ChangeCode: 등락 기호
- ChagesRatio : 전일대비 등락률
- Marcap : 시가총액(백만원)
- Stocks : 상장주식수
- MarketId : 시장기호
- Market : 시장
- Dept : 부서(한국거래소)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
pd.options.display.float_format = '{:.1f}'.format

df2020 = pd.read_csv('../data/stocks/marcap-2020.csv.gz')
df2020.head(3)

groupby에 그룹키를 지시하면, 주어진 키 column을 축으로 그룹화한 DataframeGroupBy 객체가 생성된다.

In [ ]:
df2020.groupby('Date')

여러 컬럼을 묶어 그룹으로 반환할 수 있다.

In [ ]:
df2020.groupby(['Date', 'Code'])

데이터프레임에 멀티 인덱스가 지정되어 있으면 

In [ ]:
df2 = df.set_index(['Date', 'Code'])
df2.head(3)

In [ ]:
df2.groupby('Date')

In [ ]:
df2.groupby('Date').sum()[:5]

### DataFrameGroupBy 객체 속성


In [ ]:
dg = df.groupby('Date')
dg

In [ ]:
dg.groups

그룹의 크기는 len()

In [ ]:
len(dg)

멀티 인덱스인 경우 axis 로 축을 조정할 수 있다.

In [ ]:
dg = df.groupby(['Code', 'Volume'], axis=1)
dg.groups

### 열 선택 - SeriesGroupBy

묶인 그룹은 그룹키 인덱스를 통해 얻을 수 있다. 이때 SeriesGroupBy 객체가 반환된다.

In [ ]:
dg = df.groupby(['Date'])

In [ ]:
dg.groups.keys()

In [ ]:
dg_code = dg['Code']
dg_marcap = dg['Marcap']
dg_marcap = print(dg_code, dg_marcap)

In [ ]:
dg_code.sum()[:5]

아래 같이 

In [ ]:
df['Code'].groupby(df['Date']).sum()[:5]

In [ ]:
df.groupby(['Date']).sum()[:5]

In [ ]:
df.groupby(df['Date']).sum()[:5]

### `get_group()` group 선택

그룹화 객체에서 그룹을 선택할 수 있다.

In [ ]:
dg = df.groupby(['Date'])
dg

In [ ]:
dg.get_group('20210105')[:10]

In [ ]:
dg = df.groupby(['Code'])
dg

In [ ]:
dg.get_group('005930')[:10]

여러 열로 그룹화된 경우도 가능하다.

In [ ]:
df.groupby(['Code','Name']).get_group(('005930','삼성전자'))[:10]

### 반복 처리

이터레이터를 통해 반복 처리도 가능하다. 아래는 코드별 그룹의 이름과 데이터를 사용한 예이다.

In [ ]:
dg = df[:10].groupby('Code')
for n, g in dg:
    print(n)
    print(g)

여러 그룹 키로 그룹을 지으면 그룹 이름은 튜플이 된다.

In [ ]:
dg = df[:10].groupby(['Code', 'Volume'])
for name, group in dg:
    print(name)
    print(group)

### 정렬

기본으로 그룹키를 중심으로 정렬된다. 그런데 정렬을 제거하는 `sort=False` 옵션 를 사용하면 속도향상을 위해 무시될 수 있다.

In [ ]:
%time df.groupby(['Code']).sum().head(10)

그룹키 순서로 유지 되지만, sort=False로 지시하면 원본 순서로 표시된다.

In [ ]:
%time df.groupby(['Code'], sort=False).sum().head(10)

### NA 값은 제외

그룹키에 NA가 있으면 제외되지만, 옵션으로 `dropna=False` 로 그룹키에 포함할 수 있다.

In [ ]:
df_list = [[1, 2, 3], [1, None, 4], [2, 1, 3], [1, 2, 2]]

In [ ]:
df_dropna = pd.DataFrame(df_list, columns=["a", "b", "c"])
df_dropna

In [ ]:
df_dropna.groupby('b', dropna=True).sum()

In [ ]:
df_dropna.groupby('b', dropna=False).sum()

## 집계 함수 이용

그룹화 객체는 집계 함수를 이용할 수 있다.
 - groupby.[sum, min, ...]
 - groupby.aggregate() 

집계 함수는 반환 된 개체의 차원을 줄이는 함수입니다. 몇 가지 일반적인 집계 함수는 아래 표에 나와 있습니다.

함수 | 설명
--------- | ----------------
mean() | 그룹의 평균 계산
sum() | 그룹 값의 합계 계산
size() | 그룹 크기 계산
count() | 그룹 수 계산
std() | 그룹의 표준 편차
var() | 그룹의 분산 계산
sem() | 그룹 평균의 표준 오차
describe() | 기술 통계를 생성합니다.
first() | 그룹 값의 첫 번째 계산
last() | 마지막 그룹 값 계산
nth() | n 번째 값 또는 n이 목록 인 경우 부분 집합을 사용합니다.
min() | 그룹 값의 최소값 계산
max() | 그룹 값의 최대 계산

 - 위 집계 합수는 NA 값을 제외

그룹화 객체는 집계 함수를 이용해 집합으로 처리할 수 있습니다.

In [ ]:
df.groupby(['Date', 'Code']).sum()['Marcap']

In [ ]:
dg = df.groupby(['Date'])

In [ ]:
dg.aggregate(np.sum)[:5]

여러 그룹키를 사용하면 다중인덱스 결과를 얻는다.

In [ ]:
dg = df.groupby(['Date', 'Name'])
dg.sum()[:5]

다중인덱스는 `as_index` 옵션을 사용해 인덱스 사용 여부를 변경할 수 있다.

In [ ]:
dg = df.groupby(['Date', 'Name'], as_index=False)
dg.sum()[:5]

혹은 reset_index 를 사용해서 동일한 결과를 얻을 수 있다.

In [ ]:
df.groupby(['Date', 'Name']).sum().reset_index()[:5]

각 그룹의 크기는 그룹화 객체의 `size()` 메서드를 사용할 수 있습니다.

In [ ]:
df.groupby(['Date', 'Name']).size()

In [ ]:
df.groupby(['Name']).size()

그룹화 객체의 기본 통계는 describe() 메서드

In [ ]:
dg = df.groupby(['Date'])
dg.describe()

### `.agg()` : 한 번에 여러 기능 적용

그릅화 Series 객체에는 여러 집 수행 함수, 혹은 딕트를 전달할 수 있다. 결과는 데이터프레임을 출력할 수 있다.

In [ ]:
df.groupby(['Code'])['Marcap'].agg([np.sum, np.mean, np.std])

혹은 그룹화 데이터프레임 에서는 각 열에 적용할 함수 목록을 전달해 집계 결과를 생성할 수 있다.

In [ ]:
df.groupby(['Code']).agg([np.sum, np.mean, np.std])

### 열에 다른 집계 함수 적용

aggregate 에 딕트로 다른 집계 함수를 적용할 수 있다.

In [ ]:
# df.groupby('Code').agg({'Marcap': np.sum, 'Changes': np.mean})

집계 함수를 문자로 전달할 수 있다.

In [ ]:
df.groupby('Code').agg({'Marcap': 'sum', 'Changes': 'mean'})

### Cython 최적화 집계 함수

현재 버전에서 Cython으로 sum, mean, std, sem 구현을 최적화 했다.

### 변환 `transform`

transform 메서드는 그룹화 되는 것과 동일한 인덱싱 된 개체를 반환한다. 변환함수는 다음을 충족해야 한다.
 - 그룹 청크와 크기가 같거나 그룹 청크 크기로 브로드 캐스트 할 수 있는 결과를 반환
      - 예) `grouped.transform(lambda x: x.iloc[-1)`
 - 그룹 청크에서 열 단위로 작동